#RNN 실습 기초

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Model

In [3]:
#입력값 형태 지정, 2차원 데이터를 받아 3차원 데이터를 출력하는 RNN 셀 생성
inputs = Input(shape=(1,2))
output, state = SimpleRNN(3, return_state=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
#생성된 RNN 셀을 출력
#테스트 입력값
data = np.array([[ [1,2] ]])
#출력값, 상태값 프린트
output, state = model.predict(data)
print("output: ", output)
print("state: ", state)

output:  [[ 0.8971854   0.02482994 -0.6421736 ]]
state:  [[ 0.8971854   0.02482994 -0.6421736 ]]


In [5]:
model.layers[1].weights[0]

<tf.Variable 'simple_rnn/kernel:0' shape=(2, 3) dtype=float32>

In [6]:
model.layers[1].weights[1]

<tf.Variable 'simple_rnn/recurrent_kernel:0' shape=(3, 3) dtype=float32>

In [7]:
model.layers[1].weights[2]

<tf.Variable 'simple_rnn/bias:0' shape=(3,) dtype=float32>

In [8]:
#I        [1,0,0,0]
#work     [0,1,0,0]
#at       [0,0,1,0]
#google   [0,0,0,1]
#I work at google = [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ]
#I google at work = [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]

data = np.array([
    [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ],
    [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]
])

In [10]:
#입력값의 형태 지정
inputs = Input(shape=(4,4))

output, state = SimpleRNN(3, return_state=True, return_sequences=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [12]:
#출력값, 상태값 출력
output, state = model.predict(data)

print("I work at google: ", output[0])
print("I google at work: ", output[1])

I work at google:  [[ 0.1080946  -0.37259936  0.614052  ]
 [-0.33402637 -0.83382887  0.7122181 ]
 [-0.88661206 -0.07341684  0.91741484]
 [-0.33549428  0.70423937  0.0323645 ]]
I google at work:  [[ 0.1080946  -0.37259936  0.614052  ]
 [ 0.5124081   0.3428444   0.27980915]
 [-0.48709342 -0.31369597  0.34823197]
 [-0.5156241  -0.52680933  0.6999123 ]]


In [13]:
print("I work at google: state: ", state[0])
print("I google at work: state: ", state[1])

I work at google: state:  [-0.33549428  0.70423937  0.0323645 ]
I google at work: state:  [-0.5156241  -0.52680933  0.6999123 ]


#LSTM 실습 기초

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model
import numpy as np

#입력값의 형태 지정
inputs = Input(shape=(1,2))

#RNN 셀의 속성 지정, 1차원 벡터의 출력값 지정, random_state=True: LSTM 셀의 상태값, 메모리셀을 출력하도록 지정
lstm_out, hidden_state, cell_state = LSTM(1, return_state=True)(inputs)
model = Model(inputs=inputs, outputs=[lstm_out, hidden_state, cell_state])

data = np.array([
    [ [1,0] ]
])

#출력값, 상태값, 메모리셀 출력
lstm_out, hidden_state, cell_state = model.predict(data)
print("lstm_out: ", lstm_out)
print("hidden_state: ", hidden_state)
print("cell_state: ", cell_state)

lstm_out:  [[-0.02804158]]
hidden_state:  [[-0.02804158]]
cell_state:  [[-0.08011439]]


#LSTM - 지문을 읽고 주제 분류하기

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [19]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [20]:
df.tail()

,paragraph,category
15,the perseyside outfit finished in fourth place...,sports
16,liverpool fc will return to premier league act...,sports
17,alisson signed for liverpool fc from as roma t...,sports
18,but the rankings during that run-in to new yor...,sports
19,then came the oh-so-familiar djokovic-nadal no...,sports


In [21]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


#LSTM: 데이터 전처리

In [25]:
def get_vocab_size(df):
    #데이터에 사용된 중복 없는 전체 단어 개수 리턴
    
    results = set()
    df['paragraph'].str.lower().str.split().apply(results.update)
    return len(results)

#전체 단어 갯수
vocab_size = get_vocab_size(df)

#단어를 숫자로 인코딩
paragraphs = df['paragraph'].tolist()
encoded_paragraphs = [one_hot(paragraph, vocab_size) for paragraph in paragraphs]

In [27]:
def get_max_length(df):
    #데이터에서 가장 긴 문장의 단어 갯수 리턴
    max_length = 0
    for row in df['paragraph']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

max_length = get_max_length(df)
print (max_length)

91


In [28]:
padded_paragraphs_encoding = pad_sequences(encoded_paragraphs, maxlen=max_length, padding='post')

In [29]:
padded_paragraphs_encoding

array([[382, 397, 524, ...,   0,   0,   0],
       [362, 534, 423, ...,   0,   0,   0],
       [501, 186, 238, ...,   0,   0,   0],
       ...,
       [203, 148, 295, ...,   0,   0,   0],
       [317, 129,  42, ...,   0,   0,   0],
       [283, 380, 129, ...,   0,   0,   0]])

In [31]:
categories = df['category'].tolist()
def category_encode(category):
    #분류 항목(food, sports) 수치로 변경, 원 핫 인코딩으로 변경
    
    if category == 'food':
        return [1,0]
    else:
        return [0,1]
    
encoded_category = [category_encode(category) for category in categories]

In [32]:
#원 핫 인코딩 예시
print(encoded_category[0])   #category: food
print(encoded_category[19])  #category: sports

[1, 0]
[0, 1]


In [35]:
model = Sequential()

#문맥 벡터 생성
model.add(Embedding(vocab_size, 5, input_length=max_length))

model.add(LSTM(64))

#분류 단계
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

In [36]:
train_X = np.array(padded_paragraphs_encoding)
train_y = np.array(encoded_category)

In [38]:
print('Train...')
model.fit(train_X, train_y, batch_size=10, epochs=50)

Train...
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
20/20 [==============================] - 2s 78ms/sample - loss: 0.6940 - acc: 0.5500
Epoch 2/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6933 - acc: 0.6000
Epoch 3/50
20/20 [==============================] - 0s 9ms/sample - loss: 0.6929 - acc: 0.6000
Epoch 4/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6926 - acc: 0.6000
Epoch 5/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6935 - acc: 0.5500
Epoch 6/50
20/20 [==============================] - 0s 9ms/sample - loss: 0.6923 - acc: 0.5500
Epoch 7/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6921 - acc: 0.5500
Epoch 8/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6920 - acc: 0.5500
Epoch 9/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.6918 - acc: 0.5500
Epoch 10/50
20/20 [==============================] - 0s 10ms/sample - loss: 0.691

In [40]:
score, acc = model.evaluate(train_X, train_y, verbose=2)
print('Test score; ', score)
print('Test accuracy: ', acc)

 - 0s - loss: 0.1695 - acc: 0.9500
Test score;  0.1695391684770584
Test accuracy:  0.95


In [41]:
model.summary()

#문맥 벡터 생성
#입력값은 단어들의 인덱스이고, 그 길이는 항상 91
#임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩 출력
#LSTM 셀은 64차원의 벡터의 상태값 출력

#문맥 벡터를 사용해 지문의 주제 분류
#주제 분류는 두 개의 덴즈 레이어 사용
#첫번째 Dense 레이어는 32개의 노드
#두번쨰 Dense 레이어는 2개의 노드, 이 레이어는 softmax의 입력값

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 91, 5)             2680      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                17920     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 22,746
Trainable params: 22,746
Non-trainable params: 0
_________________________________________________________________
